In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
from qetpy.utils import lowpassfilter 
from qetpy import autocuts
import cdms
#from CDMSDataCatalog import CDMSDataCatalog
import scipy.optimize as optimize
import matplotlib as mpl
import pickle as pkl
import os, sys
import ROOT as rt

/cvmfs/cdms.opensciencegrid.org/releases/centos7/V04-00/lib/python3.7/site-packages/datacat/model.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import OrderedDict, MutableMapping


Welcome to JupyROOT 6.22/00


In [9]:
Rshunt = 5.0e-3 # shunt resistance
Rfb = 5000.0 # feedback resistance
ADCperVolt = 65536.0/8.0 # the number of ADC bins per V for the readout
LoopRatio = 2.4 # SQUID turn ratio 
low_pass_filter = False
high_cutoff, fs, order = 100000, 625000, 2
trace_length = 1.6 # uS
bins = 32768
series = {
    'latest' : '23240122_150110'
}
channels=['PT','PAS1','PBS1','PCS1','PDS1','PES1','PFS1','PAS2','PBS2','PCS2','PDS2','PES2','PFS2']
rawfilepath='/sdf/group/supercdms/data/CDMS/CUTE/R37/Raw/'
filePath = '/sdf/group/supercdms/data/CDMS/CUTE/R37/Raw/23240122_150110/23240122_150110_F0003.mid.gz'

Input path is not a directory!


In [8]:
#Load detector settings
dsettings={}
for i,key in enumerate(series.keys()):
    dsettings[series[key]]=cdms.rawio.IO.getDetectorSettings(rawfilepath,series[key])

The file is: /sdf/group/supercdms/data/CDMS/CUTE/R37/Raw//23240122_150110/23240122_150110_F0001.mid.gz

Opened raw midas file: /sdf/group/supercdms/data/CDMS/CUTE/R37/Raw//23240122_150110/23240122_150110_F0001.mid.gz
INFO in DetectorConfigData: list of available detectors = 2


In [13]:

#c = 'PAS2'
data = {}
raw_data = {}
det = 'Z2'
event_number = {}
adc2ua = {}
for idx, key in enumerate(series.keys()):
#     idx = 1
#     key = list(series.keys())[idx]
#     c = 'PT'
#     mask = (TriggerType[idx] == 1) & (OFamps[series[key]][c] > 0.) & (TrigDet[idx] == 2)
#     cut = mask & (OFamps[series[key]][c] > 2e-6) & (OFamps[series[key]][c] < 20e-6)
    #event_number[c] = EventNumber[0][cut]
    t_rd = {}
    t_d = {}
    
    #event_number[key] = EventNumber[idx][cut][:10]
    for c in channels:
        if(c=='PT'):
            continue
        #event_number[c] = EventNumber[idx][cut][:10]
        if os.path.exists('./rawdata/%s_%s_selected_events.pkl'%(series[key],c)):
            t_rd[c] = pkl.load(open('./rawdata/%s_%s_selected_events.pkl'%(series[key],c), "rb"))
            print('pkl file loaded for %s'%(c))
        else:
            print('No pickle file available. Loading traces... ')
            
#             t_rd[c] = cdms.rawio.IO.getRawEvents(rawfilepath,'/'+series[key],outputFormat = 2, eventNumbers=(event_number[key]).astype(int).tolist(), 
#                                                    channelList=[c], detectorList=[2])
            t_rd[c] = cdms.rawio.IO.getRawEvents(filePath,outputFormat = 2, maxNumEvents=15, 
                                                   channelList=[c], detectorList=[2])
            pkl.dump(t_rd[c], open('./rawdata/%s_%s_selected_events.pkl'%(series[key],c), "wb"))
            print('pkl file loaded for %s'%(c))
    raw_data[key] = t_rd
    print('Raw traces [ADC] loaded : %d'%(len(raw_data[key])))
    print('Now converting into amps...')
    t_adc2uA = {}
    for c in channels:
        if(c=='PT'):
            continue
        temp = []
        DriverGain = dsettings[series[key]][det][c]['driverGain']*4.0 #Apply Driver Gain Correction
        conv2Amps = 1/(Rfb*DriverGain*LoopRatio*ADCperVolt) #Convert detector parameters to amplitude for trace
        t_adc2uA[c] = conv2Amps
        for i in range(len(t_rd[c])):
            temp.append(((t_rd[c][i])[det][c])*conv2Amps)
        t_d[c] = temp
    data[key] = t_d
    adc2ua[key] = t_adc2uA
    
    print('-------- %s : Traces coverted to Amps for channel -------------'%(key))



No pickle file available. Loading traces... 


TypeError: getRawEvents() missing 1 required positional argument: 'files_series'

In [12]:
raw_data[key]

{'PAS1': None,
 'PBS1': None,
 'PCS1': None,
 'PDS1': None,
 'PES1': None,
 'PFS1': None,
 'PAS2': None,
 'PBS2': None,
 'PCS2': None,
 'PDS2': None,
 'PES2': None,
 'PFS2': None}

In [14]:
myreader = cdms.rawio.IO.RawDataReader(filePath)

AttributeError: module 'rawio.IO' has no attribute 'RawDataReader'